In [3]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import backend as K

EPISODES = 500

In [4]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.99
        self.learning_rate = 0.001
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()

    def _huber_loss(self, target, prediction):
        # sqrt(1+error^2)-1
        error = prediction - target
        return K.mean(K.sqrt(1+K.square(error))-1, axis=-1)

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss=self._huber_loss,
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def update_target_model(self):
        # copy weights from model to target_model
        self.target_model.set_weights(self.model.get_weights())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = self.model.predict(state)
            if done:
                target[0][action] = reward
            else:
                a = self.model.predict(next_state)[0]
                t = self.target_model.predict(next_state)[0]
                target[0][action] = reward + self.gamma * t[np.argmax(a)]
            self.model.fit(state, target, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [6]:

if __name__ == "__main__":
    env = gym.make('CartPole-v1')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size)
    # agent.load("./save/cartpole-ddqn.h5")
    done = False
    batch_size = 32

    for e in range(EPISODES):
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        for time in range(500):
            env.render()
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            reward = reward if not done else -10
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                agent.update_target_model()
                print("episode: {}/{}, score: {}, e: {:.2}"
                      .format(e, EPISODES, time, agent.epsilon))
                break
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)
        # if e % 10 == 0:
        #     agent.save("./save/cartpole-ddqn.h5")

[2017-10-05 15:29:00,743] Making new env: CartPole-v1


episode: 0/500, score: 18, e: 1.0
episode: 1/500, score: 15, e: 1.0
episode: 2/500, score: 20, e: 0.99
episode: 3/500, score: 14, e: 0.98
episode: 4/500, score: 46, e: 0.97
episode: 5/500, score: 22, e: 0.96
episode: 6/500, score: 22, e: 0.95
episode: 7/500, score: 10, e: 0.94
episode: 8/500, score: 17, e: 0.93
episode: 9/500, score: 14, e: 0.92
episode: 10/500, score: 32, e: 0.91
episode: 11/500, score: 12, e: 0.9
episode: 12/500, score: 11, e: 0.9
episode: 13/500, score: 14, e: 0.89
episode: 14/500, score: 16, e: 0.88
episode: 15/500, score: 13, e: 0.87
episode: 16/500, score: 12, e: 0.86
episode: 17/500, score: 12, e: 0.85
episode: 18/500, score: 30, e: 0.84
episode: 19/500, score: 18, e: 0.83
episode: 20/500, score: 12, e: 0.83
episode: 21/500, score: 14, e: 0.82
episode: 22/500, score: 28, e: 0.81
episode: 23/500, score: 21, e: 0.8
episode: 24/500, score: 13, e: 0.79
episode: 25/500, score: 12, e: 0.79
episode: 26/500, score: 14, e: 0.78
episode: 27/500, score: 19, e: 0.77
episode

KeyboardInterrupt: 